In [1]:
import numpy as np
import meshpy.triangle

In [2]:

def create_mesh(boundary_points, max_area=0.025):
    info = meshpy.triangle.MeshInfo()
    info.set_points(boundary_points)

    def _round_trip_connect(start, end):
        return [(i, i + 1) for i in range(start, end)] + [(end, start)]

    info.set_facets(_round_trip_connect(0, len(boundary_points) - 1))

    def _needs_refinement(vertices, area):
        global points_array
        vert_origin, vert_destination, vert_apex = vertices
        bary_x = (vert_origin.x + vert_destination.x + vert_apex.x) / 3
        bary_y = (vert_origin.y + vert_destination.y + vert_apex.y) / 3
        offsets = [[point[0]-bary_x, point[1]-bary_y] for point in points_array]
        dist = [np.sqrt(point[0]**2 + point[1]**2) for point in offsets]
        boundary_dist = np.amin(dist)
        max_area = 250 * np.sqrt((boundary_dist+0.001))
        return area > max_area

    meshpy_mesh = meshpy.triangle.build(info, verbose=False, quality_meshing=True, min_angle=30, refinement_func=_needs_refinement)

    pts = np.array(meshpy_mesh.points)
    points = np.c_[pts[:, 0], pts[:, 1], np.zeros(len(pts))]

    return points, np.array(meshpy_mesh.elements)


In [3]:


if __name__ == "__main__":
    import meshio    
    surface_points = np.load('./surface.npy')
    top_points = np.load('./top.npy')
    left_points = np.load('./left.npy')
    right_points = np.load('./right.npy')
    all_points = np.vstack((surface_points, right_points, top_points, left_points))
    
    global points_array
    points_array = np.array([[boundary_point[0], boundary_point[1]] for boundary_point in surface_points])

    points, cells = create_mesh(all_points, max_area=25000)
    mesh = meshio.Mesh(points, [("triangle", cells)])
    mesh.write("mesh.vtk")